In [ ]:
# %load Prueba1.ipynb
#%run Monitoring\ Twitter.ipynb

In [25]:
import sys
import pandas as pd
import GetOldTweets3 as got
from Extract_Twitter import get_Tweets


ImportError: cannot import name 'get_Tweets' from 'Extract_Twitter' (Extract_Twitter.ipynb)

In [3]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb

In [22]:
et.get_Tweets()

AttributeError: module 'Extract_Twitter' has no attribute 'get_Tweets'

# Método Principal - TFM Data Science

In [4]:
if __name__ == '__main__':
    # Definición de variables
    tweets = creacion_DataFrame()
    calendario = calendar()
    
    #Acceso a Twitter
    api = acceso_Twitter() 
    
    #Lectura de ficheros para ir más rápido ubicados en CSVs
    #tweets = read_all_Tweets() 
    
    #Obtención por GetOldTweets
    tweets = get_Tweets(calendario)
    print(len(tweets))
    
    #Pasando el algoritmo de NLP para obtener el análisis de sentimiento
    #Generación de CSVs para el CdM

58873
